 # Парсинг данных и анализ. Кассовые сборы фильмов и бюджеты
 
Поработаем с данными о бюджетах фильмов и финансовых показателях с помощью сайта [The Numbers](https://www.the-numbers.com/movie/budgets/all).

In [ ]:
!pip install fake_useragent

In [1]:
# импортируйте нужные библиотеки
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from tqdm import tqdm
import matplotlib
%matplotlib inline 

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Парсинг данных

In [3]:
req = requests.get('https://www.the-numbers.com/movie/budgets/all')
print(req)

<Response [403]>


Мы впервые сталкиваемся с тем, что сервер не отдает нам данные по запросу. Но и из этой ситуации есть выход! Библиотека `fake_useragent`.

In [4]:
ua = UserAgent()
headers = {'User-Agent': ua.chrome}

In [5]:
req = requests.get('https://www.the-numbers.com/movie/budgets/all', headers=headers)
print(req)

<Response [200]>


In [ ]:
soup = BeautifulSoup(req.text, 'html')

In [20]:
df = pd.read_html(str(soup.find('table')))[0]

In [15]:
help(pd.read_html)

Help on function read_html in module pandas.io.html:

read_html(io: 'FilePath | ReadBuffer[str]', match: 'str | Pattern' = '.+', flavor: 'str | None' = None, header: 'int | Sequence[int] | None' = None, index_col: 'int | Sequence[int] | None' = None, skiprows: 'int | Sequence[int] | slice | None' = None, attrs: 'dict[str, str] | None' = None, parse_dates: 'bool' = False, thousands: 'str | None' = ',', encoding: 'str | None' = None, decimal: 'str' = '.', converters: 'dict | None' = None, na_values=None, keep_default_na: 'bool' = True, displayed_only: 'bool' = True) -> 'list[DataFrame]'
    Read HTML tables into a ``list`` of ``DataFrame`` objects.
    
    Parameters
    ----------
    io : str, path object, or file-like object
        String, path object (implementing ``os.PathLike[str]``), or file-like
        object implementing a string ``read()`` function.
        The string can represent a URL or the HTML itself. Note that
        lxml only accepts the http, ftp and file url proto

In [21]:
df

,Unnamed: 0,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,1,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$684,075,767","$2,319,591,720"
1,2,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,788,912,285"
2,3,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
3,4,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,395,316,979"
4,5,"May 17, 2023",Fast X,"$340,000,000","$145,960,660","$713,979,164"
...,...,...,...,...,...,...
95,96,"Sep 4, 2020",Mulan,"$200,000,000",$0,"$69,973,540"
96,97,"Jul 2, 2021",The Tomorrow War,"$200,000,000",$0,"$19,220,000"
97,98,"Jul 13, 2022",The Gray Man,"$200,000,000",$0,"$451,178"
98,99,"Jun 29, 2011",Transformers: Dark of the Moon,"$195,000,000","$352,390,543","$1,123,794,079"


Спарсим все данные по ссылке выше.

In [26]:
main = pd.DataFrame()

for i in tqdm(range(1, 65)):
    req = requests.get(f'https://www.the-numbers.com/movie/budgets/all/{i}01', headers=headers)
    soup = BeautifulSoup(req.text, 'html')
    table = soup.find('table')
    df1 = pd.read_html(str(table))[0]
    main = pd.concat([main, df1])

100%|███████████████████████████████████████████| 64/64 [03:29<00:00,  3.27s/it]


In [27]:
df = pd.concat([df, main])

In [28]:
df = df.set_index('Unnamed: 0')

In [29]:
df.shape

(6434, 5)

### EDA

* Сколько строк и столбцов содержит набор данных?
* Присутствуют ли значения NaN?
* Есть ли дублирующиеся строки?
* Какие типы данных столбцов?

In [30]:
# ваш код
df.isna().values.any()

False

In [31]:
df.duplicated().values.any()

False

In [ ]:
?df.dropna

In [ ]:
?df.drop_duplicates

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6434 entries, 1 to 6434
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ReleaseDate       6434 non-null   object
 1   Movie             6434 non-null   object
 2   ProductionBudget  6434 non-null   object
 3   DomesticGross     6434 non-null   object
 4   WorldwideGross    6434 non-null   object
dtypes: object(5)
memory usage: 301.6+ KB


### Перевод данных

Переводим столбцы в нужные типы данных

In [33]:
my_cols = list(df.columns)
my_cols

['ReleaseDate', 'Movie', 'ProductionBudget', 'DomesticGross', 'WorldwideGross']

In [ ]:
cols = ['ProductionBudget', 'DomesticGross', 'WorldwideGross']

for col in cols:
    df[col] = df[col].str.replace('$', '')
    df[col] = df[col].str.replace(',', '')
    df[col] = pd.to_numeric(df[col])

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6434 entries, 1 to 6434
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ReleaseDate       6434 non-null   object
 1   Movie             6434 non-null   object
 2   ProductionBudget  6434 non-null   int64 
 3   DomesticGross     6434 non-null   int64 
 4   WorldwideGross    6434 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 301.6+ KB


### Описательная статистика

* Какой средний бюджет фильмов в наборе данных?
* Какой средний мировой доход фильмов?
* Какие минимальные показатели мирового и локального дохода фильмов?
* Какие самый высокий бюджет и самый высокий мировой доход среди фильмов?
* Какой доход принесли фильмы с самым низким и самым высоким бюджетом?

In [51]:
# ваш код
pd.options.display.float_format = '{:,.2f}'.format

df.describe()

,ProductionBudget,DomesticGross,WorldwideGross
count,"6,434.00","6,434.00","6,434.00"
mean,"32,493,886.84","41,954,185.26","93,284,689.47"
std,"43,755,883.27","71,929,639.07","186,045,067.85"
min,86.00,0.00,0.00
25%,"5,000,000.00","1,084,873.75","3,747,858.75"
50%,"17,000,000.00","16,083,016.50","27,147,670.00"
75%,"40,000,000.00","51,566,931.75","96,671,373.00"
max,"460,000,000.00","936,662,225.00","2,923,706,026.00"


In [ ]:
# ваш код

### Невышедшие фильмы

* Определите, какие фильмы еще не вышли в прокат
* Создайте другой датаcет под названием data, который не будет включать эти фильмы.

In [116]:
# ваш код

### Фильмы, потерявшие деньги

* Какой процент фильмов, в которых затраты на производство превысили мировой доход?

In [ ]:
# ваш код